<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/Classifiers_with_BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Important and useful sources:
https://stackoverflow.com/questions/61631446/attributeerror-list-object-has-no-attribute-lower-with-countvectorizer

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import nltk
import gensim
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
np.random.seed(42) #The random seed helps in the reproduction of the results

In [4]:
dataset=pd.read_excel('ARPs_and_ProgrammingPosts.xlsx')

In [4]:
dataset.head()

,Post,Class,Tokens,W2V_Vect,Tf-IDF_vect,BoW_vect,fastText_vect,GloVe_vect
0,Here's the architecture:\n<ol>\n<li>Input comp...,1,NaN,NaN,NaN,NaN,NaN,NaN
1,I am confused about the architecture of Razor ...,1,NaN,NaN,NaN,NaN,NaN,NaN
2,I have some spring boot microservices and I wa...,1,NaN,NaN,NaN,NaN,NaN,NaN
3,I'm trying to properly design an application a...,1,NaN,NaN,NaN,NaN,NaN,NaN
4,I heard that for .NET8 Microsoft gifted us wit...,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dataset.shape #I have 14918 instances/posts and 2 attributes/columns in the dataset.

(14932, 2)

In [ ]:
dataset.Class

0        1
1        1
2        1
3        1
4        1
        ..
14927    0
14928    0
14929    0
14930    0
14931    0
Name: Class, Length: 14932, dtype: int64

In [ ]:
dataset.columns

Index(['Post', 'Class'], dtype='object')

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14932 entries, 0 to 14931
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Post    14932 non-null  object
 1   Class   14932 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 233.4+ KB


In [ ]:
dataset[dataset['Class'] ==1].count()

Post     7466
Class    7466
dtype: int64

In [ ]:
dataset[dataset['Class'] ==0].count()

Post     7466
Class    7466
dtype: int64

In [ ]:
dataset.Post[304]

"<p>I am new to microservices , and currently designing a SASS based multi tenant application .\nnow i have 3 services , one for auth related activities , one for user ,roles and rbac and one for tenant  all the services have their own respective DB's.\nnow since its a multi tenant application. i am not able to decide in which DB do i store the tenant - User related mapping .\ndo i create a seperate tables for mapping tenets and store in the tenant service DB or do i take the tenant ID from the service and store the tenant related mapping in the user service Database .</p>\n<p>Thanks .</p>\n"

# Tokenization

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)
print("-----------------------------------------------")

import string
punctuations = string.punctuation
print(punctuations)

{"'re", 'does', '‘d', 'been', 'namely', 'an', 'then', 'did', 'same', 'yet', 'it', '’d', 'still', 'them', 'who', 'last', 'many', 'me', 'she', 'against', 'to', 'whereupon', 'thereby', 'make', 'upon', 'becoming', 'above', 'anyway', 'herself', 'should', 'of', 'he', 'whose', 'four', 'another', 'bottom', 'across', 'hers', 'fifteen', 'from', 'yourselves', 'those', 'top', 'everything', "'ve", 'become', 'has', 'just', 'we', 'such', 'other', 'therefore', 'sometimes', 'cannot', 'twenty', 'you', 'well', 'eleven', 'always', 'nowhere', 'behind', 'thus', 'the', 'moreover', 'whence', '‘re', 'his', 'meanwhile', 'due', 'during', 'were', 'side', 'several', 'about', 'really', 'own', 'quite', 'though', 'herein', 'because', 'every', 'onto', 'everyone', 'was', 'they', 'although', 'out', 'one', 'any', 'therein', 'keep', 'among', 'its', 'there', 'without', 'together', 'anywhere', 'their', 'various', 'over', 'see', 'latterly', 'used', 'only', 'or', 'third', 'under', 'ten', 'show', 'either', 'even', 'again', 'th

In [5]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [6]:
dataset['Tokens'] = dataset['Post'].apply(spacy_tokenizer)

In [7]:
#dataset.head()

In [7]:
#def Bow_vectorizer(word)
x = dataset['Tokens'].map(' '.join)
y = dataset['Class']

In [8]:
#dataset.head()

# Features Engineering with BoW

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.33, random_state=125)

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train)
X_train_vect = vect.transform(X_train).toarray()
X_test_vect = vect.transform(X_test).toarray()

In [ ]:
#dataset['BoW_vect'] = dataset['Tokens']

# Naive Bayes with BoW

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [14]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train_vect, y_train)

y_pred = nb_model.predict(X_test_vect)

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82      2544
           1       0.81      0.80      0.81      2384

    accuracy                           0.81      4928
   macro avg       0.81      0.81      0.81      4928
weighted avg       0.81      0.81      0.81      4928



# kNN with BoW

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [13]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train_vect, y_train)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
y_pred = knn_model.predict(X_test_vect)

In [11]:
print(classification_report(y_test, y_pred))

# Decison Tree with BoW

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
dt_model = DecisionTreeClassifier()
dt_model = dt_model.fit(X_train_vect, y_train)

In [14]:
y_pred = dt_model.predict(X_test_vect)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      2544
           1       0.94      0.96      0.95      2384

    accuracy                           0.95      4928
   macro avg       0.95      0.95      0.95      4928
weighted avg       0.95      0.95      0.95      4928



# Logistic Regression with BoW

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
lg_model = LogisticRegression(C=5, penalty='l1', solver='liblinear')
lg_model = lg_model.fit(X_train_vect, y_train)

In [11]:
y_pred = lg_model.predict(X_test_vect)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      2544
           1       0.94      0.96      0.95      2384

    accuracy                           0.95      4928
   macro avg       0.95      0.95      0.95      4928
weighted avg       0.95      0.95      0.95      4928



#SVM with BoW

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_model = SVC()
svm_model = svm_model.fit(X_train_vect, y_train)

In [11]:
y_pred = svm_model.predict(X_test_vect)
print(classification_report(y_test, y_pred))

# Neural Net

In [ ]:
import tensorflow as tf

In [ ]:
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
  ax1.plot(history.history['loss'], label='loss')
  ax1.plot(history.history['val_loss'], label='val_loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Binary crossentropy')
  ax1.grid(True)

  ax2.plot(history.history['accuracy'], label='accuracy')
  ax2.plot(history.history['val_accuracy'], label='val_accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.grid(True)

  plt.show()

In [ ]:
def train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
  nn_model = tf.keras.Sequential([
      tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(10,)),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(num_nodes, activation='relu'),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy',
                  metrics=['accuracy'])
  history = nn_model.fit(
    X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0
  )

  return nn_model, history